In [1]:
import warnings
warnings.filterwarnings("ignore") 
# CUSTOMARY IMPORTS
import torch
import torchvision
from  torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np


%matplotlib inline

# TRAINING HYPERPARAMETERS: 
n_epochs = 5           # How many passes through the training data  
batch_size = 64  # Training batch size usually in [1,256]

learning_rate = 0.01   # Learning rate for optimizer like SGD usually in [0.001, 0.1]

random_seed = 1  

torch.manual_seed(random_seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./dataset', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True)

testset = torchvision.datasets.CIFAR10(
    root='./dataset', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

In [4]:
class Depthwise_Conv(nn.Module):
    def __init__(self, in_planes, kernel_size=3, stride=1, padding=1, bias=False):
        super(Depthwise_Conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_planes, in_planes, kernel_size=kernel_size, stride=stride, \
                      padding=padding, groups=in_planes, bias=bias),
            nn.BatchNorm2d(in_planes),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        out = self.conv(x)
        return out

class Pointwise_Conv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size=1, padding=0, bias=False):
        super(Pointwise_Conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, padding=padding, bias=bias),
            nn.BatchNorm2d(out_planes),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        out = self.conv(x)
        return out
    
class Block(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1):
        super(Block, self).__init__()
        self.conv1 = Depthwise_Conv(in_planes, stride=stride)
        self.conv2 = Pointwise_Conv(in_planes, out_planes)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        return out

class MobileNet(nn.Module):
    net_info = [(64, 1), (128,2), (128, 1), (256,2), (256, 1), (512,2), (512,1), 
                (512, 1), (512,1), (512,1), (512,1), (1024,2), (1024,1)]

    def __init__(self, in_planes, out_planes):
        super(MobileNet, self).__init__()
        self.conv1 = Pointwise_Conv(in_planes, 32, kernel_size=3, padding=1, bias=False);
        self.layers = self._make_layers(in_planes=32)
        self.linear = nn.Linear(1024, out_planes)

    def _make_layers(self, in_planes):
        layers = []
        for layer_info in self.net_info:
            out_planes, stride = layer_info
            layers.append(Block(in_planes, out_planes, stride))
            in_planes = out_planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layers(out)
        out = F.avg_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [5]:
from tqdm import tqdm

in_channel = 3
classes = 10
net = MobileNet(in_channel, classes).to(device)
net.load_state_dict(torch.load("ciphar.pth"))
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train(True)
    processBar = tqdm(trainloader,unit = 'step')
    train_loss = 0
    total_correct = 0
    total = 0
    total_size = 0
    for batch_idx, (X_image, y_label) in enumerate(processBar):
        X_image, y_label = X_image.to(device), y_label.to(device)
        optimizer.zero_grad()
        outputs = net(X_image)
        loss = criterion(outputs, y_label)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        total_size += y_label.size(0)
        predictions = torch.argmax(outputs, dim = 1)
        total = y_label.size(0)
        correct = torch.sum(predictions == y_label)
        total_correct += correct
        processBar.set_description("[%d/%d] Loss: %.4f, Acc: %.4f" % 
                                    (epoch,n_epochs,loss.item(),100.*correct/total))

In [9]:
n_epochs = 20
for epoch in range(1, n_epochs + 1):
    train(epoch)
    scheduler.step()
torch.save(net.state_dict(),'./ciphar2.pth')


Epoch: 1


[1/20] Loss: 0.0480, Acc: 100.0000: 100%|███████████████████████████████████████████████████| 782/782 [00:30<00:00, 26.02step/s]



Epoch: 2


[2/20] Loss: 0.0217, Acc: 100.0000: 100%|███████████████████████████████████████████████████| 782/782 [00:30<00:00, 25.95step/s]



Epoch: 3


...

[18/20] Loss: 0.9549, Acc: 75.0000: 100%|███████████████████████████████████████████████████| 782/782 [00:30<00:00, 25.90step/s]



Epoch: 19


[19/20] Loss: 0.3386, Acc: 93.7500: 100%|███████████████████████████████████████████████████| 782/782 [00:30<00:00, 25.93step/s]



Epoch: 20


[20/20] Loss: 0.0900, Acc: 100.0000: 100%|██████████████████████████████████████████████████| 782/782 [00:30<00:00, 26.00step/s]


In [10]:
def test():
    net.eval()
    test_loss = 0
    total_size = 0
    total_correct = 0
    processBar = tqdm(testloader,unit = 'step')
    with torch.no_grad():
        for batch_idx, (X_image, y_label) in enumerate(processBar):
            X_image, y_label = X_image.to(device), y_label.to(device)
            outputs = net(X_image)
            loss = criterion(outputs, y_label)

            test_loss += loss.item()
            total_size += y_label.size(0)
            predictions = torch.argmax(outputs, dim = 1)
            total = y_label.size(0)
            correct = torch.sum(predictions == y_label)
            total_correct += correct
            processBar.set_description("[%d/%d] Loss: %.4f, Acc: %.4f" % 
                                        (epoch,n_epochs,loss.item(),100.*correct/total))
    processBar.set_description("[%d/%d] Loss: %.4f, Acc: %.4f" % 
                                        (epoch,n_epochs,test_loss,100.*total_correct/total_size))

In [11]:
test()

[20/20] Loss: 0.2919, Acc: 93.7500: 100%|███████████████████████████████████████████████████| 157/157 [00:02<00:00, 65.79step/s]


1. Yes, neural networks can be used with un supervised learning and dimension reduction, this can be achieved through the use of auto-encoders, which is usually used to pre-train models.
2. Neural networks can learn and model both non-linear and complex relationships, and can manage the relationship between inputs and outputs in a relatively simple way. Can be trained continuously to improve their performances.
3. When training time is not sufficient and computing resources are not enough, they can't be trained properly and might perform very badly.
4. Neural networks need large amount of data to be trained, and if data-set is sufficiently large it is a good candidate for classification problems.